# Different types of MultiClass SVMs
The dataset used is CIFAR-10

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
import os
from six.moves import cPickle as pickle
import time

In [65]:
train_df = pd.read_csv("train.csv")
train_df.Activity = train_df.Activity.map({"LAYING":0, "SITTING":1, "STANDING":2, "WALKING":3, "WALKING_DOWNSTAIRS":4, "WALKING_UPSTAIRS":5} )
test_df = pd.read_csv("test.csv")
test_df.Activity = test_df.Activity.map({"LAYING":0, "SITTING":1, "STANDING":2, "WALKING":3, "WALKING_DOWNSTAIRS":4, "WALKING_UPSTAIRS":5} )

num_classes = 6

train_np = train_df.to_numpy()
X_train = train_np[:,:-1]
y_train = train_np[:,-1]

test_np = test_df.to_numpy()
X_test = test_np[:,:-1]
y_test = test_np[:,-1]

X_merged = X_train
X_merged = np.concatenate((X_merged, X_test), axis=0)
mean = np.mean(X_merged)
std = np.std(X_merged)
X_train -= mean
X_test -= mean
X_train /= std
X_test /= std

print(X_train)
print(X_test) 
print(y_train)
print(y_test)

[[ 0.8062065   0.48177982  0.36350036 ...  0.6920938   0.44151751
   1.55343246]
 [ 0.79552911  0.48585891  0.37335772 ...  0.69245961  0.44604468
   1.55343246]
 [ 0.79682547  0.48264846  0.38392253 ...  0.69282557  0.45150528
   1.55343246]
 ...
 [ 0.79024439  0.48522866  0.45580745 ...  0.76478156  0.54596103
  32.01320328]
 [ 0.80733     0.48330399  0.33684759 ...  0.76193241  0.52971053
  32.01320328]
 [ 0.87229261  0.49004708  0.28896632 ...  0.76232764  0.54163748
  32.01320328]]
[[ 0.77321885  0.4786382   0.48770415 ...  0.79382989  0.44219878
   2.60376939]
 [ 0.80351995  0.48926957  0.37801877 ...  0.79860039  0.41497435
   2.60376939]
 [ 0.79244741  0.47573382  0.37899647 ...  0.79727708  0.4197553
   2.60376939]
 ...
 [ 0.87067784  0.53468698  0.38147917 ...  0.79139073  0.69339931
  25.71118173]
 [ 0.75264911  0.52249197  0.40173914 ...  0.78120544  0.70009979
  25.71118173]
 [ 0.66445585  0.48373099  0.35917999 ...  0.78031746  0.70066748
  25.71118173]]
[2. 2. 2. ... 5. 

In [66]:
df = pd.read_csv("Iris.csv")
df.label = df.label.map({"Iris-setosa":0,"Iris-versicolor":1,"Iris-virginica":2})
df = df.to_numpy()

num_classes = 3
mask = int(df.shape[0]*0.9)

X_train = df[:mask,:-1]
y_train = df[:mask,-1]

X_test = df[mask:,:-1]
y_test = df[mask:,-1]

print(X_train.shape)
print(X_test.shape)


(135, 4)
(15, 4)


### 0. Default -> Practice
taking too long :(

In [3]:
'''tic = time.time()
clf = svm.SVC()
clf.fit(X_train, y_train)
toc = time.time()

print("Training time => %ds" % (toc-tic))

tic = time.time()
y_test_pred = clf.predict(X_test)
accuracy = np.mean(y_test_pred == y_test)
toc = time.time()

print("Training time => %ds" % (toc-tic))
print("Accuracy => %d%" % (accuracy*100))'''

'tic = time.time()\nclf = svm.SVC()\nclf.fit(X_train, y_train)\ntoc = time.time()\n\nprint("Training time => %ds" % (toc-tic))\n\ntic = time.time()\ny_test_pred = clf.predict(X_test)\naccuracy = np.mean(y_test_pred == y_test)\ntoc = time.time()\n\nprint("Training time => %ds" % (toc-tic))\nprint("Accuracy => %d%" % (accuracy*100))'

### 1. One Against All
We need 10 binary svms
- For each clas (0-9), we create a new y_train 
    - this has values = 1 for the class and 0 for the others
    - Train with this new y_train
    
- For testing:
    - Test each image against each svm, and record highest value in y_test_pred

In [67]:
# training:
# create svms list
OAASvms = []

# for each class:
tic = time.time()
for i in range(num_classes):
    # get new y_train (x_train is same) and train new svm
    new_y_train = []
    for j in y_train:
        if j == i:
            new_y_train.append(1)
        else:
            new_y_train.append(0)
                                
    tic1 = time.time()
    new_svm = svm.SVC(kernel='linear')
    new_svm.fit(X_train, new_y_train)
    toc1 = time.time()
    
    print("Time to train for class ", i, "  => %ds" % (toc1-tic1))
    
    # append to list
    OAASvms.append(new_svm)
toc = time.time()

print("Total training time => %ds" % (toc-tic))

Time to train for class  0   => 0s
Time to train for class  1   => 0s
Time to train for class  2   => 0s
Total training time => 0s


In [68]:
df = pd.read_csv("Iris.csv")
df.label = df.label.map({"Iris-setosa":0,"Iris-versicolor":1,"Iris-virginica":2})
df = df.to_numpy()

num_classes = 3
mask = int(df.shape[0]*0.9)

X_train = df[:mask,:-1]
y_train = df[:mask,-1]

X_test = df[mask:,:-1]
y_test = df[mask:,-1]

print(X_train.shape)
print(X_test.shape)


(135, 4)
(15, 4)


In [69]:
# predict:
y_pred = []

# for each image
tic = time.time()
for sample in X_test:
    # for each svm
    maxval = -100
    predictedclass = -1;
    for idx, this_svm in enumerate(OAASvms):
        val = np.dot(this_svm.coef_, sample) + this_svm.intercept_
        
        if val > maxval:
            maxval = val
            predictedclass = idx
    
    y_pred.append(predictedclass)
toc = time.time()

# check overall accuracy
accuracy = np.mean(y_pred == y_test)
print("Testing time => %ds" % (toc-tic))
print("Accuracy     => %f" % (accuracy*100))

Testing time => 0s
Accuracy     => 100.000000


In [70]:
df = pd.read_csv("Iris.csv")
df.label = df.label.map({"Iris-setosa":0,"Iris-versicolor":1,"Iris-virginica":2})
df = df.to_numpy()

num_classes = 3
mask = int(df.shape[0]*0.9)

X_train = df[:mask,:-1]
y_train = df[:mask,-1]

X_test = df[mask:,:-1]
y_test = df[mask:,-1]

print(X_train.shape)
print(X_test.shape)


(135, 4)
(15, 4)


### 2. One Against One
We need (10*9)/2 binary svms
- For each pair of classes a and b, we create a classifier:  
    - this has values = 1 for the class i and 0 for the class j
    - other classes are removed from consideration
    - Train with this new y_train
    
- For testing:
    - Test each image against each svm, and record highest value in y_test_pred

In [71]:
# training:
# create svms list
rows, cols = (num_classes, num_classes) 
OAOSvms = [[None]*cols]*rows 

# for each class:
tic = time.time()
for a in range(1,num_classes):
    for b in range(a):
        # get new y_train (x_train is same) and train new svm
        new_y_train = []
        new_x_train_indices = []
        for idx, j in enumerate(y_train):
            if j == a:
                new_y_train.append(1)
                new_x_train_indices.append(idx)
            elif j == b:
                new_y_train.append(0)
                new_x_train_indices.append(idx)
        new_x_train = X_train[new_x_train_indices]
                
        tic1 = time.time()
        new_svm = svm.SVC(kernel='linear')
        new_svm.fit(new_x_train, new_y_train)
        toc1 = time.time()

        print("Time to train for class ",a,b, "  => %ds" % (toc1-tic1))

        # append to list
        OAOSvms[a][b] = new_svm
toc = time.time()

print("Total training time => %ds" % (toc-tic))

Time to train for class  1 0   => 0s
Time to train for class  2 0   => 0s
Time to train for class  2 1   => 0s
Total training time => 0s


In [72]:
# predict:
y_pred = []


# for each image
tic = time.time()
'''for idx, sample in enumerate(X_test):
    if (idx == 2):
        break
    
    # for each svm
    maxval = -100
    count = np.zeros(num_classes);
    

    
    # max count is predicted class
    predictedclass = np.where(count == np.amax(count))
    
    y_pred.append(predictedclass)'''
# voting by every svm
vote_counts = np.zeros((X_test.shape[0],num_classes))
for a in range(1,num_classes):
    for b in range(a):
        this_svm = OAOSvms[a][b]
        pred = this_svm.predict(X_test)
        
        b_indices = np.where(pred == 0)
        a_indices = np.where(pred == 1)
        
        a_additive = np.zeros(num_classes)
        b_additive = np.zeros(num_classes)
        a_additive[a] = 1;
        b_additive[b] = 1;

        vote_counts[b_indices] += b_additive
        vote_counts[a_indices] += a_additive
        
        print(a,b,vote_counts[0], pred[0])
        

# take max over columns $$
print(vote_counts[0])
print(y_test[0])
y_pred = []
for i in vote_counts:
    pos = np.where(i == np.amax(i))
    y_pred.append(pos)

toc = time.time()

# check overall accuracy
accuracy = np.mean(y_pred == y_test)
print("Testing time => %fs" % (toc-tic))
print("Accuracy     => %f" % (accuracy*100))

1 0 [0. 1. 0.] 1
2 0 [0. 1. 1.] 1
2 1 [0. 1. 2.] 1
[0. 1. 2.]
2.0
Testing time => 0.004097s
Accuracy     => 93.333333


In [73]:

# iterative function to test abocve implementation
def getClassOAO(sample, target):
    votes = np.zeros(num_classes)
    
    for a in range(1,num_classes):
        for b in range(a):
            this_svm = OAOSvms[a][b]
            pred = this_svm.predict(sample)
            
            if pred == 1:
                votes[a] += 1
            else:
                votes[b] += 1
                
    return np.where(votes == np.amax(votes))

# predict:
y_pred = []

# for each image
tic = time.time()
for idx, sample in enumerate(X_test):
    y_pred.append(getClassOAO(sample.reshape(1, -1), y_test[idx]))

toc = time.time()

# check overall accuracy
accuracy = np.mean(y_pred == y_test)
print("Testing time => %fs" % (toc-tic))
print("Accuracy     => %f" % (accuracy*100))

Testing time => 0.004813s
Accuracy     => 93.333333


### 3. DAG 
We need (10*9)/2 binary svms again
- For each pair of classes a and b, we create a classifier:  (same as OAO)
    - this has values = 1 for the class i and 0 for the class j
    - other classes are removed from consideration
    - Train with this new y_train
    
- For testing: 
    - Test 1xn, then if it favours discard 1 and test against 1x(n-1), etc.

In [76]:
# iterative function to eliminate one class each time
def getClass(sample, target):
    curr_b = 0
    curr_a = num_classes-1
    
    # stopping condition is when we reach only one number
    while (curr_a != curr_b):
        this_svm = OAOSvms[curr_a][curr_b];
        pred = this_svm.predict(sample)
        
        # if favours a, remove b
        if (pred == 1):
            curr_b += 1;
        else:
            curr_a -= 1;
        
    return curr_a

# predict:
y_pred_DAG = []

# for each image
tic = time.time()
for idx, sample in enumerate(X_test):
    y_pred_DAG.append(getClass(sample.reshape(1, -1), y_test[idx]))

toc = time.time()

# check overall accuracy
accuracy = np.mean(y_pred_DAG == y_test)
print("Testing time => %ds" % (toc-tic))
print("Accuracy     => %f" % (accuracy*100))

Testing time => 0s
Accuracy     => 93.333333
